Prediction on the Cifar 10 case, with data augmentation and batch normalization

In [1]:
import tensorflow as tf
import numpy as np 
import matplotlib.pyplot as plt
from tensorflow.keras.layers import Input, Conv2D, Dense, Flatten, MaxPooling2D, Dropout, BatchNormalization
from tensorflow.keras.models import Model
from tensorflow.keras.preprocessing.image import ImageDataGenerator

cifar10 = tf.keras.datasets.cifar10

(x_train, y_train),(x_test, y_test) = cifar10.load_data()
x_train, x_test = x_train/255, x_test/255
y_train, y_test = y_train.flatten(), y_test.flatten()
print("x_train.shape:", x_train.shape)
print("y_train.shape:", y_train.shape)

170498071/170498071 [==============================] - 25s 0us/step
x_train.shape: (50000, 32, 32, 3)
y_train.shape: (50000,)


In [2]:
# number of classes
cl_num = len(set(y_train))
print("There are: ", cl_num, " classes")

There are:  10  classes


In [ ]:
import matplotlib.pyplot as plt

# model 
i = Input(shape=x_train[0].shape) # shape of first elem
x = Conv2D(32, (3,3), activation='relu', padding='same')(i)
x = BatchNormalization()(x)
x = Conv2D(32, (3,3), activation='relu', padding='same')(i)
x = BatchNormalization()(x)
x = MaxPooling2D((2,2))(x)

x = Conv2D(64, (3,3), activation='relu', padding='same')(x)
x = BatchNormalization()(x)
x = Conv2D(64, (3,3), activation='relu', padding='same')(x)
x = BatchNormalization()(x)
x = MaxPooling2D((2,2))(x)

x = Conv2D(128, (3,3), activation='relu', padding='same')(x)
x = BatchNormalization()(x)
x = Conv2D(128, (3,3), activation='relu', padding='same')(x)
x = BatchNormalization()(x)
x = MaxPooling2D((2,2))(x)

x = Flatten()(x)
x = Dropout(0.4)(x)
x = Dense(1024, activation='relu')(x)
x = Dropout(0.2)(x)
x = Dense(cl_num, activation='softmax')(x)

model = Model(i, x)
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metric=["accuracy"])

batch_size = 32
data_generator = tf.keras.preprocessing.image.ImageDataGenerator(width_shift_range=0.1, height_shift_range=0.1, horizontal_flip=True)
train_generator = data_generator.flow(x_train, y_train, batch_size)
steps_per_epoch = x_train.shape[0] // batch_size
trained_model = model.fit_generator(train_generator, validation_data=(x_test, y_test),steps_per_epoch = steps_per_epoch, epochs = 15)

# save model
model.save('\\Fashion and Cifar\\fashion_model.h5')
print('Model Saved!')

# plotting the results
plt.plot(trained_model.history['accuracy'], label='acc')
plt.plot(trained_model.history['val_accuracy'], label='val_acc')
plt.title('Accuracy plot')
plt.legend()
plt.show()

plt.plot(trained_model.history['loss'], label='acc')
plt.plot(trained_model.history['val_loss'], label='val_acc')
plt.title('Loss plot')
plt.legend()
plt.show()

In [ ]:
# plot misclassified examples to see if they make sense
pre_test = model.predict(x_test).argmax(axis=1)
wrong_class_idx = np.where( pre_test != y_test)[0]

labels = '''airplane automobile bird cat deer dog frog horse ship truck'''.split()
i = np.random.choice(wrong_class_idx)
plt.imshow(x_test[i])
plt.title("True Label: %s Predicted: %s" % (labels[y_test[i]], labels[pre_test[i]]))